In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import cv2 as cv       
import serial        
import time
import os
import threading
import sys

# Diretório para salvar as imagens
DIR_IMG = "TT1/"
os.makedirs(DIR_IMG, exist_ok=True)

# Configurações do GRBL e porta serial
PORT = "COM4"         # Porta de comunicação com Arduino/GRBL
BAUDRATE = 115200     # Taxa de transferência - padrão GRBL

# Dados das plantas
ID_PLANT = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B010', 'B11', 'B12']
POS_X_PLANT = [-132.313, -132.313, -132.313, -132.313, -132.313, -132.313, -782.313, -782.313, -782.313, -782.313, -782.313, -782.313]
POS_Y_PLANT = [-1810.775, -1510.775, -1110.775, -810.775, -460.775, -110.775, -1835.775, -1460.775, -1155.350, -768.613, -463.100, -79.275]

# Inicializa câmera e conexão serial (serão definidos na função init_system)
cam = None
grbl = None

# Tempo de espera para conclusão dos movimentos (em segundos)
MOVE_DELAY = 10

# ================= Funções de Hardware =================

def iniciar_camera(camera_id=0, largura=1920, altura=1080):
    print("Iniciando câmera...")
    cam_local = cv.VideoCapture(camera_id)
    if not cam_local.isOpened():
        messagebox.showerror("Erro", "Não foi possível abrir a câmera. Verifique a conexão.")
        sys.exit(1)
    cam_local.set(3, largura)
    cam_local.set(4, altura)
    return cam_local

def iniciar_serial(porta, baudrate):
    print("Iniciando comunicação com GRBL...")
    try:
        grbl_local = serial.Serial(porta, baudrate, timeout=1)
        time.sleep(2)  
        grbl_local.write(b"\r\n\r\n") 
        time.sleep(2)
        grbl_local.flushInput() 
        print("Conectado ao GRBL na porta:", porta)
        return grbl_local
    except serial.SerialException as e:
        messagebox.showerror("Erro", f"Erro ao conectar na porta {porta}: {e}")
        sys.exit(1)

def send_grbl(cmd, delay=0.1):
    if grbl is None:
        print("GRBL não inicializado!")
        return
    comando = cmd + "\r\n"
    grbl.write(comando.encode())
    time.sleep(delay)
    while grbl.inWaiting() > 0:
        response = grbl.readline().decode().strip()
        print("GRBL:", response)

def capturar_imagem(indice):
    if cam is None:
        print("Câmera não inicializada!")
        return
    ret, frame = cam.read()
    if not ret:
        print("Erro ao capturar imagem!")
        return
    frame_resized = cv.resize(frame, (640, 360))
    cv.imshow('Imagem Capturada', frame_resized)
    nome_arquivo = os.path.join(DIR_IMG, f"{ID_PLANT[indice]}.jpg")
    cv.imwrite(nome_arquivo, frame)
    cv.waitKey(30)
    print(f"Imagem da planta {ID_PLANT[indice]} salva em {nome_arquivo}")

# ================= Lógica de Movimentação e Captura =================

def processar_planta(indice, velocidade):
    try:
        # Define a velocidade de deslocamento
        cmd_vel = f'G1 F{velocidade}'
        print("Configurando velocidade:", cmd_vel)
        send_grbl(cmd_vel)
        
        # Comando de movimentação para a planta selecionada
        cmd_move = f"G1 X{POS_X_PLANT[indice]} Y{POS_Y_PLANT[indice]}"
        print("Movendo para:", cmd_move)
        send_grbl(cmd_move)
        
        # Aguarda o tempo de deslocamento
        time.sleep(MOVE_DELAY)
        
        # Captura a imagem
        capturar_imagem(indice)
        
        # Opcional: Retorna para a posição inicial (X0 Y0)
        print("Retornando para a posição inicial (X0 Y0)...")
        send_grbl('G0 X0 Y0')
        time.sleep(MOVE_DELAY)
        messagebox.showinfo("Concluído", f"Processo da planta {ID_PLANT[indice]} concluído!")
    except Exception as e:
        messagebox.showerror("Erro", f"Erro durante o processo da planta {ID_PLANT[indice]}: {e}")

def executar_planta(indice):
    try:
        velocidade = int(speed_entry.get())
    except ValueError:
        messagebox.showerror("Erro", "Velocidade inválida. Por favor, insira um número inteiro.")
        return
    thread = threading.Thread(target=processar_planta, args=(indice, velocidade))
    thread.start()

# ================= Funções de Inicialização e Fechamento =================

def init_system():
    global cam, grbl
    cam = iniciar_camera()
    grbl = iniciar_serial(PORT, BAUDRATE)
    # Teste inicial de captura
    capturar_imagem(0)

def close_system():
    if grbl is not None:
        grbl.close()
    if cam is not None:
        cam.release()
    cv.destroyAllWindows()
    root.destroy()

# ================= Interface Gráfica (Tkinter) =================

# Criação da janela principal
root = tk.Tk()
root.title("Interface CNC + Camera USB")
root.geometry("500x600")

# Frame para configurações gerais
frame_config = ttk.LabelFrame(root, text="Configurações")
frame_config.pack(padx=10, pady=10, fill="x")

# Campo para definir a velocidade (F)
ttk.Label(frame_config, text="Velocidade (F):").grid(row=0, column=0, padx=5, pady=5, sticky="e")
speed_entry = ttk.Entry(frame_config, width=10)
speed_entry.insert(0, "7000")  
speed_entry.grid(row=0, column=1, padx=5, pady=5, sticky="w")

# Frame para os botões das plantas
frame_plants = ttk.LabelFrame(root, text="Seleção de Planta")
frame_plants.pack(padx=10, pady=10, fill="both", expand=True)

# Cria botões para cada planta (B01 a B12) organizados em duas colunas
for i, plant_id in enumerate(ID_PLANT):
    btn = ttk.Button(frame_plants, text=plant_id, command=lambda idx=i: executar_planta(idx))
    # Distribui os botões em 2 colunas
    row = i // 2
    col = i % 2
    btn.grid(row=row, column=col, padx=10, pady=5, sticky="ew")

# Botão para encerrar o sistema
btn_close = ttk.Button(root, text="Fechar Sistema", command=close_system)
btn_close.pack(pady=10)

# Inicializa o sistema (câmera e GRBL) antes de iniciar a interface
init_system()

# Inicia a interface gráfica
root.protocol("WM_DELETE_WINDOW", close_system)
root.mainloop()